# Set up cell magic/games dataset

In [1]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [2]:
import numpy as np
import pandas as pd

games = pd.read_csv(r"C:\Users\mckid\Downloads\games_may2024_cleaned.csv")

# Joining all reviews datasets

In [3]:
%%skip
import os

reviewdir = '/Users/shobhanashreedhar/SteamReviews2024'

review_files = {}

In [4]:
%%skip
for f in os.listdir(reviewdir):
    fpath = os.path.join(reviewdir, f)
    key = int(f[:-4])
    review_files[key] = fpath
    df = pd.read_csv(fpath)
    df['AppID'] = key
    df.to_csv(fpath)
    print('appended AppID to', fpath)

In [5]:
%%skip
games = games[games['AppID'].isin(review_files.keys())]

In [6]:
%%skip
import dask.dataframe as dd

reviewfiles = '/Users/shobhanashreedhar/SteamReviews2024/*.csv'
ddf = dd.read_csv(reviewfiles, assume_missing=True, dtype={'steam_china_location': 'object'})
reviews = ddf.compute()
reviews.to_csv('reviews.csv', index=False)


# Set up datasets

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
#%%skip
dtypedict = {'Unnamed: 0': 'Int64', 'recommendationid': 'Int64', 'language': 'object', 'timestamp_created': 'Int64', 'timestamp_updated': 'Int64',
            'voted_up': 'Int64', 'votes_up': 'Int64', 'votes_funny': 'Int64', 'weighted_vote_score': 'float', 'comment_count': 'Int64',
             'steam_purchase': 'Int64', 'received_for_free': 'Int64', 'written_during_early_access': 'Int64', 'hidden_in_steam_china': 'Int64',
             'steam_china_location': 'object', 'author_steamid': 'Int64','author_num_games_owned': 'Int64', 'author_num_reviews': 'Int64', 
             'author_playtime_forever': 'float', 'author_playtime_last_two_weeks': 'float', 'author_playtime_at_review': 'float', 
             'author_last_played': 'Int64', 'AppID': 'Int64', 'Unnamed: 0.1': 'Int64'}
reviews = pd.read_csv(r"C:\Users\mckid\Downloads\reviews.csv", dtype=dtypedict)

In [22]:
games

,AppID,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,29018,753,6078,352,1362469,"{'FPS': 90076, 'Shooter': 64786, 'Multiplayer'...",87,8071426,80,56072
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,590582,"{'Survival': 14611, 'Shooter': 12477, 'Battle ...",58,2365012,68,19517
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,35557,1327,1163,977,668192,"{'Free to Play': 59559, 'MOBA': 19966, 'Multip...",81,2247365,71,23832
3,271590,Grand Theft Auto V,2015-04-13,17,0.00,1,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,12898,590,6910,235,133571,"{'Open World': 31998, 'Action': 23177, 'Multip...",87,1643791,92,16181
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,19.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,16213,578,3016,331,68162,"{'FPS': 9689, 'PvP': 9073, 'e-sports': 9026, '...",85,1091695,77,10566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83641,1745960,Circuit Breaker,2021-10-21,0,4.99,1,Circuit Breaker is a turn-based strategy game ...,Circuit Breaker is a turn-based strategy game ...,"In the cyber-future of 2009, the most popular ...",NaN,...,0,0,0,0,0,"{'Strategy': 72, 'Casual': 48, 'PvE': 47, 'Sin...",-1,-1,-1,-1
83642,1888920,Chess vs Chat,2022-08-09,0,3.99,0,Chess Vs Chat allows streamers to host chess m...,Chess Vs Chat allows streamers to host chess m...,Allow your chatters to actively participate an...,NaN,...,0,0,0,0,0,"{'Indie': 31, 'Strategy': 21, 'Puzzle': 11, 'M...",-1,-1,-1,-1
83643,2166420,Orbtangle,2022-10-28,0,1.99,0,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,NaN,...,0,0,0,0,0,"{'Puzzle': 54, 'Colorful': 46, 'Relaxing': 44,...",-1,-1,-1,-1
83644,1456260,Flashlight,2021-10-25,0,2.99,0,Play with Friends Play with or against your fr...,Play with Friends Play with or against your fr...,"Salvage scrap, build defenses, and rank-up to ...",NaN,...,0,0,0,0,0,"{'Top-Down Shooter': 138, 'Rogue-lite': 132, '...",-1,-1,-1,-1


In [23]:
#%%skip
reviews

,Unnamed: 0,recommendationid,language,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,...,steam_china_location,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,AppID,Unnamed: 0.1
0,0,147937429,english,1696875102,1717510986,1,3,0,0.526800,0,...,NaN,76561199550893216,35,23,59161.0,4738.0,58753.0,1717541057,10,<NA>
1,1,166664841,russian,1717510100,1717510100,1,0,0,0.000000,0,...,NaN,76561199161536896,24,11,436.0,71.0,385.0,1717512997,10,<NA>
2,2,166664763,russian,1717510009,1717510009,1,0,0,0.000000,0,...,NaN,76561198046827632,0,7,23750.0,7.0,23743.0,1717510490,10,<NA>
3,3,166663001,turkish,1717508182,1717508182,0,0,0,0.000000,0,...,NaN,76561199374468448,32,4,361.0,19.0,356.0,1717508513,10,<NA>
4,4,166658743,brazilian,1717503385,1717503385,1,1,0,0.523810,0,...,NaN,76561198018922960,9,1,1497.0,0.0,1497.0,1478272196,10,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127525533,10,55089874,german,1568377930,1568377930,1,1,0,0.525862,0,...,NaN,76561197999112224,0,11,355.0,0.0,275.0,1660071500,999990,<NA>
127525534,11,55018278,koreana,1567947602,1567947602,1,1,0,0.000000,0,...,NaN,76561198088855088,159,55,160.0,0.0,160.0,1567947383,999990,<NA>
127525535,12,55003469,koreana,1567867308,1567867308,1,7,2,0.589744,0,...,NaN,76561198052389904,0,298,59.0,0.0,59.0,1567867162,999990,<NA>
127525536,13,54917785,koreana,1567339322,1567339322,1,5,2,0.513882,0,...,NaN,76561198009493312,0,10,479.0,0.0,173.0,1665410232,999990,<NA>


In [24]:
#%%skip
rev_small = reviews.sample(n=100000)
rev_small.to_csv('rev_small.csv')

In [25]:
#%%skip
del reviews

In [32]:
rev_small = pd.read_csv('rev_small.csv', dtype=dtypedict)

In [33]:
rev_small

,Unnamed: 0.2,Unnamed: 0,recommendationid,language,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,...,steam_china_location,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,AppID,Unnamed: 0.1
0,43897552,25566,147113530,schinese,1695801198,1695801198,0,0,0,0.000000,...,NaN,76561199517876608,0,1,116.0,0.0,116.0,1695795991,2338770,<NA>
1,20870000,668,87185275,english,1613938054,1613938054,1,1,0,0.000000,...,NaN,76561198093920480,0,7,2351.0,0.0,831.0,1655721957,1379560,<NA>
2,19905291,6827,103234225,russian,1637526903,1637526903,1,0,0,0.000000,...,NaN,76561197971873760,516,62,591.0,0.0,136.0,1638122875,1338770,6827
3,34936375,71577,8254936,english,1387904684,1387904684,1,1,0,0.521739,...,NaN,76561198049816512,335,3,228.0,0.0,228.0,1373857857,206440,<NA>
4,52533650,14174,8086049,brazilian,1387320358,1387320358,1,1,0,0.518519,...,NaN,76561198086670544,0,1,2102.0,0.0,139.0,1595009046,259080,14174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3278998,38667,129373656,spanish,1672092870,1672092870,1,0,0,0.000000,...,NaN,76561198143000032,141,25,8749.0,0.0,8749.0,1635500733,1063730,<NA>
99996,52239603,169877,60878924,english,1577669315,1577669315,0,0,0,0.000000,...,NaN,76561198118717344,0,64,16439.0,0.0,6886.0,1687788029,255710,<NA>
99997,65070282,36188,21477423,english,1456503218,1456503218,1,0,0,0.000000,...,NaN,76561197974132912,0,31,407.0,0.0,330.0,1457106730,322500,<NA>
99998,74124905,27132,58076168,english,1574831602,1574831602,1,0,0,0.000000,...,NaN,76561198120374960,0,31,2039.0,0.0,797.0,1692606197,383980,<NA>


In [34]:
rev_small.drop(['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [35]:
rev_small

,recommendationid,language,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,...,hidden_in_steam_china,steam_china_location,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,AppID
0,147113530,schinese,1695801198,1695801198,0,0,0,0.000000,0,1,...,1,NaN,76561199517876608,0,1,116.0,0.0,116.0,1695795991,2338770
1,87185275,english,1613938054,1613938054,1,1,0,0.000000,0,1,...,0,NaN,76561198093920480,0,7,2351.0,0.0,831.0,1655721957,1379560
2,103234225,russian,1637526903,1637526903,1,0,0,0.000000,0,1,...,0,NaN,76561197971873760,516,62,591.0,0.0,136.0,1638122875,1338770
3,8254936,english,1387904684,1387904684,1,1,0,0.521739,1,1,...,0,NaN,76561198049816512,335,3,228.0,0.0,228.0,1373857857,206440
4,8086049,brazilian,1387320358,1387320358,1,1,0,0.518519,2,0,...,0,NaN,76561198086670544,0,1,2102.0,0.0,139.0,1595009046,259080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,129373656,spanish,1672092870,1672092870,1,0,0,0.000000,0,0,...,1,NaN,76561198143000032,141,25,8749.0,0.0,8749.0,1635500733,1063730
99996,60878924,english,1577669315,1577669315,0,0,0,0.000000,0,1,...,0,NaN,76561198118717344,0,64,16439.0,0.0,6886.0,1687788029,255710
99997,21477423,english,1456503218,1456503218,1,0,0,0.000000,0,1,...,0,NaN,76561197974132912,0,31,407.0,0.0,330.0,1457106730,322500
99998,58076168,english,1574831602,1574831602,1,0,0,0.000000,0,1,...,0,NaN,76561198120374960,0,31,2039.0,0.0,797.0,1692606197,383980


In [36]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83646 entries, 0 to 83645
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AppID                     83646 non-null  int64  
 1   name                      83643 non-null  object 
 2   release_date              83646 non-null  object 
 3   required_age              83646 non-null  int64  
 4   price                     83646 non-null  float64
 5   dlc_count                 83646 non-null  int64  
 6   detailed_description      83480 non-null  object 
 7   about_the_game            83458 non-null  object 
 8   short_description         83537 non-null  object 
 9   reviews                   10283 non-null  object 
 10  header_image              83646 non-null  object 
 11  website                   39760 non-null  object 
 12  support_url               42081 non-null  object 
 13  support_email             73067 non-null  object 
 14  window

In [37]:
rev_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   recommendationid                100000 non-null  Int64  
 1   language                        100000 non-null  object 
 2   timestamp_created               100000 non-null  Int64  
 3   timestamp_updated               100000 non-null  Int64  
 4   voted_up                        100000 non-null  Int64  
 5   votes_up                        100000 non-null  Int64  
 6   votes_funny                     100000 non-null  Int64  
 7   weighted_vote_score             100000 non-null  float64
 8   comment_count                   100000 non-null  Int64  
 9   steam_purchase                  100000 non-null  Int64  
 10  received_for_free               100000 non-null  Int64  
 11  written_during_early_access     100000 non-null  Int64  
 12  hidden_in_steam_c

In [40]:
combined = pd.merge(games, rev_small, on='AppID', how='left')

In [41]:
combined

,AppID,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,written_during_early_access,hidden_in_steam_china,steam_china_location,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,0,1,NaN,76561199078182992,12,1,292385.0,0.0,227896.0,1696423067
1,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,0,0,NaN,76561199103424576,14,2,60266.0,64.0,2172.0,1698086044
2,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,0,1,NaN,76561199407131328,1,1,12456.0,0.0,5273.0,1697482797
3,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,0,1,NaN,76561199248259248,0,3,5828.0,89.0,634.0,1698750403
4,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,0,0,NaN,76561199115639840,0,9,13373.0,3.0,324.0,1698528259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173243,1745960,Circuit Breaker,2021-10-21,0,4.99,1,Circuit Breaker is a turn-based strategy game ...,Circuit Breaker is a turn-based strategy game ...,"In the cyber-future of 2009, the most popular ...",NaN,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>
173244,1888920,Chess vs Chat,2022-08-09,0,3.99,0,Chess Vs Chat allows streamers to host chess m...,Chess Vs Chat allows streamers to host chess m...,Allow your chatters to actively participate an...,NaN,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>
173245,2166420,Orbtangle,2022-10-28,0,1.99,0,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,Orbtangle is a simplistic puzzle game which re...,NaN,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>
173246,1456260,Flashlight,2021-10-25,0,2.99,0,Play with Friends Play with or against your fr...,Play with Friends Play with or against your fr...,"Salvage scrap, build defenses, and rank-up to ...",NaN,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>


In [42]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173248 entries, 0 to 173247
Data columns (total 67 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   AppID                           173248 non-null  int64  
 1   name                            173245 non-null  object 
 2   release_date                    173248 non-null  object 
 3   required_age                    173248 non-null  int64  
 4   price                           173248 non-null  float64
 5   dlc_count                       173248 non-null  int64  
 6   detailed_description            172508 non-null  object 
 7   about_the_game                  172430 non-null  object 
 8   short_description               173137 non-null  object 
 9   reviews                         43576 non-null   object 
 10  header_image                    173248 non-null  object 
 11  website                         118589 non-null  object 
 12  support_url     